# Stress Level Prediction – LSTM, CNN & NCF Models
Updated notebook including LSTM & CNN deep learning models.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, LSTM, Conv1D, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

df = pd.read_csv('dataset_prediksi_stres_1000_balanced.csv')
df.head()

2025-11-23 14:59:19.467942: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-23 14:59:20.421376: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-23 14:59:23.344323: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


,employee_id,department,workload,work_life_balance,team_conflict,management_support,work_environment,stress_level,label
0,EMP_SYN_1_0120,Operations,3,4,4,0,7,30,1
1,EMP_SYN_2_0127,IT,10,4,6,7,10,60,2
2,EMP_SYN_0_0209,Finance,10,10,3,8,0,20,0
3,EMP_SYN_0_0309,Marketing,0,3,5,5,6,17,0
4,EMP_SYN_2_0074,HR,4,7,3,4,2,49,2


## Preprocessing

In [2]:
def classify_level(x):
    if x <= 20: return 0
    elif x <= 40: return 1
    else: return 2

df['label'] = df['stress_level'].apply(classify_level)

le_dept = LabelEncoder()
df['dept_enc'] = le_dept.fit_transform(df['department'])

features = ['workload','work_life_balance','team_conflict','management_support','work_environment']
X = df[features]
y = df['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
y_train_cat = to_categorical(y_train, 3)
y_test_cat = to_categorical(y_test, 3)

X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

df.head()

,employee_id,department,workload,work_life_balance,team_conflict,management_support,work_environment,stress_level,label,dept_enc
0,EMP_SYN_1_0120,Operations,3,4,4,0,7,30,1,4
1,EMP_SYN_2_0127,IT,10,4,6,7,10,60,2,2
2,EMP_SYN_0_0209,Finance,10,10,3,8,0,20,0,0
3,EMP_SYN_0_0309,Marketing,0,3,5,5,6,17,0,3
4,EMP_SYN_2_0074,HR,4,7,3,4,2,49,2,1


## LSTM Model

In [3]:
lstm_inp = Input(shape=(1, X_train.shape[1]))
x = LSTM(64)(lstm_inp)
x = Dense(32, activation='relu')(x)
out = Dense(3, activation='softmax')(x)

lstm_model = Model(lstm_inp, out)
lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_lstm = lstm_model.fit(X_train_lstm, y_train_cat, epochs=50, batch_size=16, validation_split=0.2, verbose=0)

y_pred_lstm = np.argmax(lstm_model.predict(X_test_lstm), axis=1)
print("LSTM Accuracy:", accuracy_score(y_test, y_pred_lstm))
print(classification_report(y_test, y_pred_lstm))

E0000 00:00:1763884764.623975  436159 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1763884764.639972  436159 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
LSTM Accuracy: 0.52
              precision    recall  f1-score   support

           0       0.70      0.56      0.62        81
           1       0.49      0.56      0.52        63
           2       0.38      0.43      0.40        56

    accuracy                           0.52       200
   macro avg       0.52      0.51      0.51       200
weighted avg       0.54      0.52      0.53       200



## CNN Model

In [4]:
cnn_inp = Input(shape=(X_train.shape[1], 1))
x = Conv1D(32, kernel_size=2, activation='relu')(cnn_inp)
x = MaxPooling1D()(x)
x = Conv1D(64, kernel_size=2, activation='relu')(x)
x = GlobalAveragePooling1D()(x)
x = Dense(32, activation='relu')(x)
out = Dense(3, activation='softmax')(x)

cnn_model = Model(cnn_inp, out)
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_cnn = cnn_model.fit(X_train_cnn, y_train_cat, epochs=50, batch_size=16, validation_split=0.2, verbose=0)

y_pred_cnn = np.argmax(cnn_model.predict(X_test_cnn), axis=1)
print("CNN Accuracy:", accuracy_score(y_test, y_pred_cnn))
print(classification_report(y_test, y_pred_cnn))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
CNN Accuracy: 0.535
              precision    recall  f1-score   support

           0       0.72      0.57      0.63        81
           1       0.48      0.48      0.48        63
           2       0.42      0.55      0.48        56

    accuracy                           0.54       200
   macro avg       0.54      0.53      0.53       200
weighted avg       0.56      0.54      0.54       200



## Neural Collaborative Filtering (NCF) Model

In [5]:
user_enc = LabelEncoder()
df['user_enc'] = user_enc.fit_transform(df['employee_id'])
num_users = df['user_enc'].nunique()
num_items = df['dept_enc'].nunique()

X_user = df['user_enc']
X_item = df['dept_enc']
y2 = df['label']

X_user_train, X_user_test, X_item_train, X_item_test, y2_train, y2_test = train_test_split(
    X_user, X_item, y2, test_size=0.2, random_state=42)

y2_train_cat = to_categorical(y2_train, 3)
y2_test_cat = to_categorical(y2_test, 3)

user_in = Input(shape=(1,))
item_in = Input(shape=(1,))

user_emb = Embedding(num_users, 16)(user_in)
item_emb = Embedding(num_items, 16)(item_in)

u = Flatten()(user_emb)
i = Flatten()(item_emb)

concat = Concatenate()([u, i])
x = Dense(32, activation='relu')(concat)
x = Dense(16, activation='relu')(x)
out = Dense(3, activation='softmax')(x)

ncf = Model([user_in, item_in], out)
ncf.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

ncf.fit([X_user_train, X_item_train], y2_train_cat, epochs=50, batch_size=16, verbose=0)

y2_pred = np.argmax(ncf.predict([X_user_test, X_item_test]), axis=1)
print("NCF Accuracy:", accuracy_score(y2_test, y2_pred))
print(classification_report(y2_test, y2_pred))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
NCF Accuracy: 0.35
              precision    recall  f1-score   support

           0       0.41      0.27      0.33        81
           1       0.34      0.29      0.31        63
           2       0.32      0.54      0.40        56

    accuracy                           0.35       200
   macro avg       0.36      0.36      0.35       200
weighted avg       0.36      0.35      0.34       200

